In [ ]:
# Importation de la bibliothèque pandas pour la manipulation de données
import pandas as pd

# Importation de la bibliothèque matplotlib.pyplot pour la création de graphiques
import matplotlib.pyplot as plt

# Importation de la bibliothèque missingno pour la visualisation des données manquantes
import missingno as msno

# Importation de la bibliothèque re pour les expressions régulières 
import re

In [ ]:
# Chargement des données depuis le fichier Excel 
donnees_fifa = pd.read_excel("./fifa_data.xlsx")

In [ ]:
# Affichage du DataSet
donnees_fifa.tail()

In [ ]:
def profilage_de_data(donnees):
    # Affichage de la matrice de valeurs manquantes avec le nom des colonnes
    msno.matrix(donnees)

    # Récupération des noms de colonnes
    noms_colonnes = donnees.columns

    # Personnalisation de l'affichage en ajoutant les noms de colonnes dans l'en-tête
    plt.title("Matrice de valeurs manquantes avec noms de colonnes")
    plt.xlabel("Colonnes")
    plt.ylabel("Lignes")
    plt.xticks(ticks=range(len(noms_colonnes)), labels=noms_colonnes, rotation=90)

    # Afficher le graphique
    return plt.show()

In [ ]:
# Appel de la fonction de profilage de données avec le DataFrame donnees_fifa
profilage_de_data(donnees_fifa)

In [ ]:
# Suppression des colonnes spécifiées
donnees_fifa = donnees_fifa.drop(columns=["Unnamed: 89","Joined","Contract Valid Until", "Release Clause", "Photo", 
                                          "Flag", "Club Logo","Real Face", "Loaned From", "Special","Unnamed: 0",
                                          "Jersey Number", "Value", "LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW",
                                          "LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", 
                                          "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB","Club"], axis=1)

In [ ]:
# Affichage de la nouvelle matrice de valeurs manquantes
profilage_de_data(donnees_fifa)

In [ ]:
# Recherche des doublons
donnees_fifa.duplicated().sum()

# Suppression des lignes vides
donnees = donnees_fifa.dropna(subset=['Preferred Foot',"Position"]) 

# Suppression des lignes dont les postes ne sont pas mentionnés
donnees = donnees.drop(donnees[donnees["Position"] == "No"].index)

In [ ]:
# Affichage de la nouvelle matrice de valeurs manquantes
msno.matrix(donnees)

In [ ]:
# Affichage des différents postes avec leurs occurrences
occurrences = donnees['Position'].value_counts()

In [ ]:
# Substitution des postes du milieu de terrain
donnees.loc[:, "Position"] = donnees["Position"].replace({"CAM":"CM", "CDM":"CM", "RCM":"RM", "RAM":"RM","LCM":"LM", "LAM":"LM"})

# Substitution des postes de la défense 
donnees.loc[:, "Position"] = donnees["Position"].replace({"RCB": "RB", "RWB": "RB","LCB": "LB", "LWB": "LB"})

# Substitution des postes de l'attaque 
donnees.loc[:, "Position"] = donnees["Position"].replace({"CF": "ST","RS": "ST", "LS": "ST","LF": "ST", "RF": "ST"})

In [ ]:
# Création de la colonne Groupe de poste
donnees['GroupPositions'] = ""

# Remplissage de la colonne de façon conditionnelle
donnees.loc[donnees["Position"]== "GK" , "GroupPositions"] = "Goalkeeper"
donnees.loc[donnees["Position"]== "ST" , "GroupPositions"] = "Striker"
donnees.loc[donnees["Position"].isin(["CB","RB","LB"]), "GroupPositions"] = "Defender"
donnees.loc[donnees["Position"].isin(["CM","RM","LM"]), "GroupPositions"] = "Midfielder"
donnees.loc[donnees["Position"].isin(["RDM","LDM"]), "GroupPositions"] = "Defensive Midfielder"
donnees.loc[donnees["Position"].isin(["RW","LW"]), "GroupPositions"] = "Winger"

In [ ]:
# Renommage des abréviations de la colonne Position
position_mapping = {
    'ST': 'Striker',
    'CF': 'Center Forward',
    'LW': 'Left Winger',
    'RW': 'Right Winger',
    'CM': 'Central Midfielder',
    'CDM': 'Central Defensive Midfielder',
    'CAM': 'Central Attacking Midfielder',
    'LM': 'Left Midfielder',
    'RM': 'Right Midfielder',
    'CB': 'Center Back',
    'LB': 'Left Back',
    'RB': 'Right Back',
    'LWB': 'Left Wing Back',
    'RWB': 'Right Wing Back',
    'LS': 'Left Striker',
    'RS': 'Right Striker',
    'LCM': 'Left Central Midfielder',
    'RCM': 'Right Central Midfielder',
    'LDM': 'Left Defensive Midfielder',
    'RDM': 'Right Defensive Midfielder',
    'LF': 'Left Forward',
    'RF': 'Right Forward',
    'LAM': 'Left Attacking Midfielder',
    'RAM': 'Right Attacking Midfielder',
}

donnees['Position'] = donnees['Position'].map(position_mapping)

In [ ]:
# Suppression de l'unité de mesure et conversion en float pour la colonne Crossing
donnees['Crossing'] = pd.to_numeric(donnees['Crossing'], errors='coerce', downcast='integer')

In [ ]:
# Suppression de l'unité de mesure et conversion en float pour la colonne Weight
donnees["Weight"] = donnees["Weight"].astype(str).str.replace("lbs", "").astype(float)

# Conversion du poids en kilogrammes
donnees["Weight"] = donnees["Weight"] * 0.454

In [ ]:
# Suppression de l'unité de mesure et conversion en float pour la colonne Height
donnees["Height"] = donnees["Height"].astype(str).str.extract('(\d+)').astype(float)

# Conversion de la hauteur en mètres
donnees["Height"] = donnees["Height"] * 0.3048

In [ ]:
# Création de la colonne Imc et ajout des données
donnees['Imc'] = donnees['Weight'] / donnees['Height'] 
donnees['Imc']

In [ ]:
# Repération des données incorrectes dans la colonne Wage
i = 0
for element in donnees["Wage"]:
    if re.fullmatch("(€[0-9]{1}K|€[0-9]{3}K|€[0-9]{2}K|€[0-9]{1})", str(element)):
        i += 1
    else:
        print(element)

# Extraction des chiffres du motif recherché et changement du type de la colonne 
donnees["Wage"] = donnees["Wage"].str.extract('(\d+)').astype(int) * 1000

In [ ]:
# Repération des données incorrectes dans la colonne International Reputation
i = 0
for element in donnees["International Reputation"]:
    if re.fullmatch("([0-9]{1})", str(element)):
        i += 1
    else:
        print(element)

# Changement du type de la colonne
donnees['International Reputation'] = pd.to_numeric(donnees['International Reputation'], errors='coerce').fillna(0).astype(int)

# Changement du type de la colonne
donnees['International Reputation'] = donnees['International Reputation'].astype(int)

In [ ]:
# Repération des données incorrectes dans la colonne Body Type
i = 0
for element in donnees["Body Type"]:
    if element in ["Normal","Lean","Stocky"] : 
        i += 1
    else: 
        print(element)

# Correction des irrégularités
donnees.loc[~donnees["Body Type"].isin(["Normal","Lean","Stocky"]), "Body Type"] = "Normal"

In [ ]:
# Exportation du dataset clean
donnees.to_excel("../DataSet/data_fifa_clean.xlsx", index=False)